# 📈 股票漲跌預測模型：K-Nearest Neighbors (KNN) & LightGBM

### 1. 模型概述 (Model Overview)
本程式實作KNN機器學習模型，旨在利用多維度技術指標、全球宏觀數據及相關市場數據來預測台積電 (2330) 的股價漲跌。
- **KNN**: 基於距離的分類算法，對於特徵間的非線性關係具備良好的捕捉能力。

### 2. 特徵工程說明 (Feature Engineering)
本模型使用的主要特徵包括：
- **價量指標**: Open, High, Low, Close, Volume, Adj Close.
- **技術指標**: MA (移動平均), RSI, MACD 系列指標。
- **市場聯動**: 台積電 ADR (ADR_Close), 道瓊指數 (DJI_Close), 波動率指數 (VIX_Close)。
- **預處理**: 針對 KNN 模型進行 `StandardScaler` 標準化，以消除特徵量綱差異。

### 3. 環境需求 (Requirements)
執行前請確保安裝以下套件：
- `pandas`, `numpy`, `scikit-learn`, `matplotlib`

### 4. 資料夾規範 (File Paths)
- **輸入**: `../../data/processed/train.csv` & `test.csv`
- **輸出**: `../../data/results/KNN_pred.csv` (預測結果將供後續 Stacking 使用)

---

# 📊 第一階段：資料獲取與環境初始化 (Data Acquisition & Setup)

本專案旨在建構一個穩健的台積電 (2330) 股價漲跌預測模型。在開始進行特徵工程與模型訓練之前，我們先進行資料的載入與基礎結構確認。

### 1. 資料來源說明 (Data Sources)
我們整合了多個維度的金融數據，時間跨度涵蓋 2010 至 2025 年，總計十六年的交易數據：
* **台股盤後資料**: 包含 2330 的價量資訊（Open, Close, High, Low, Volume）及籌碼面特徵。
* **技術指標**: 原始資料已包含 MA, RSI, MACD, K/D 等關鍵量化指標。
* **美股聯動數據**: 整合 NASDAQ、S&P 500 (SPX)、道瓊工業指數 (DJI) 及費城半導體指數 (SOX)。
* **外部情緒指標**: 台積電 ADR 走勢與 VIX 波動率指數。

### 2. 資料集切割原則 (Dataset Splitting)
* **訓練集 (Training Set)**: 使用2010-2024的歷史數據，建立穩定的市場特徵模型。
* **測試集 (Test Set)**: 使用2025的數據作為 Out-of-Sample 測試，驗證模型在未見過資料下的泛化能力。

### 3. 目標設定
本階段將確保資料讀取路徑正確，並對時間欄位進行標準化轉換，為後續的特徵增強 (Feature Enhancement) 奠定基礎。

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_raw = pd.read_csv("../../../data/processed/train.csv")
test_raw  = pd.read_csv("../../../data/processed/test.csv")

In [3]:
# 如果 Date 是字串，轉成 datetime
if 'Date' in train_raw.columns:
    train_raw['Date'] = pd.to_datetime(train_raw['Date'])
    test_raw['Date']  = pd.to_datetime(test_raw['Date'])

In [4]:
print(train_raw.columns.tolist())

['Date', 'Capacity', 'Turnover', 'Open', 'High', 'Low', 'Close', 'Change', 'Transaction', 'MA5', 'MA10', 'MA5_Capacity', 'BR5', 'BR10', 'High-Low', 'Open-Close', 'EMA5', 'EMA10', 'K', 'D', 'MACD', 'MACD_signal', 'MACD_hist', 'RSI14', 'STD20', 'ADX', 'RSI', 'DJI', 'NASDAQ', 'SOX', 'SPX', 'ADR', 'twclose', 'twopen', 'Movement', 'rate']


## 🛠️ 第二階段：資料處理與特徵工程 (Data Preprocessing & Feature Engineering)

本階段將原始金融數據轉換為模型可理解的特徵。我們不僅保留了基礎的價量資訊，更透過特徵增強函式 `enhance_features` 擴展了多維度的分析維度。

### 1. 核心特徵分類
* **市場基礎特徵**: 包含 Open, High, Low, Close, Volume 等原始價量數據。
* **技術分析指標 (Technical Indicators)**: 
    * 趨勢指標：MA5, MA10, EMA, MACD。
    * 動能指標：RSI, K, D, ADX。
    * 波動指標：STD20, Bollinger Bands (布林通道)。
* **全球市場聯動 (Global Macro)**: 整合美股關鍵指數（DJI, NASDAQ, SOX, SPX）及台積電 ADR 走勢。
* **自定義增強特徵**: 
    * **滯後項 (Lags)**: 考慮過去 1 至 10 日的價格與動能變化，捕捉時間序列的慣性。
    * **型態分析**: K 棒實體大小、上下影線比例及光頭光腳線辨識。
    * **連續性指標**: 計算連續上漲與下跌天數以偵測市場情緒。

In [5]:
def enhance_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    特徵工程增強函式：
    將原始股價與指標數據轉換為更具預測力的特徵（包含動量、型態、均線交叉、美股聯動等）。
    """
    df = df.copy()
    
    # 確保按日期排序：時間序列模型必須確保過去的資料不會參考到未來（防止資料洩漏）
    if 'Date' in df.columns:
        df = df.sort_values('Date').reset_index(drop=True)

    # ============= 1. 基礎價格與技術指標衍生特徵 =============
    # 計算當日漲跌幅與震幅
    df['Price_momentum'] = df['Close'] / df['Open'] - 1
    df['High_Low_range'] = (df['High'] - df['Low']) / df['Close']

    # 均線交叉特徵：判斷短期趨勢是否強於中期
    df['MA5_MA10_cross']   = (df['MA5'] > df['MA10']).astype(int)
    df['Close_above_MA5']  = (df['Close'] > df['MA5']).astype(int)
    df['Close_above_MA10'] = (df['Close'] > df['MA10']).astype(int)

    # RSI 強弱判斷：定義超買(>70)與超跌(<30)區間
    df['RSI_oversold']   = (df['RSI'] < 30).astype(int)
    df['RSI_overbought'] = (df['RSI'] > 70).astype(int)

    # MACD 趨勢與黃金交叉判斷
    df['MACD_above_signal'] = (df['MACD'] > df['MACD_signal']).astype(int)
    df['MACD_golden_cross'] = (
        (df['MACD'] > df['MACD_signal']) &
        (df['MACD'].shift(1) <= df['MACD_signal'].shift(1))
    ).astype(int)
    df['MACD_positive'] = (df['MACD'] > 0).astype(int)

    # KD 指標交叉與區間判斷 (判斷短期轉折)
    df['K_above_D'] = (df['K'] > df['D']).astype(int)
    df['K_D_golden_cross'] = (
        (df['K'] > df['D']) & (df['K'].shift(1) <= df['D'].shift(1))
    ).astype(int)
    df['K_D_dead_cross'] = (
        (df['K'] < df['D']) & (df['K'].shift(1) >= df['D'].shift(1))
    ).astype(int)
    df['KD_oversold']   = ((df['K'] < 20) & (df['D'] < 20)).astype(int)
    df['KD_overbought'] = ((df['K'] > 80) & (df['D'] > 80)).astype(int)

    # 美股指數聯動：計算道瓊(DJI)、標普500(SPX)、那斯達克(NASDAQ)的當日變動率
    df['DJI_change']    = df['DJI'].pct_change()
    df['SPX_change']    = df['SPX'].pct_change()
    df['NASDAQ_change'] = df['NASDAQ'].pct_change()

    # 相對強弱特徵：個股相對於大盤(twclose)的超額報酬
    df['Stock_vs_TW'] = df['Close'].pct_change() - df['twclose'].pct_change()

    # 滯後項特徵 (Lags)：捕捉歷史數據的慣性（考慮過去 1~10 天的漲跌、漲幅與 RSI）
    for lag in [1, 2, 3, 4, 5, 6, 7, 10]:
        df[f'Movement_lag{lag}'] = df['Movement'].shift(lag)
        df[f'Change_lag{lag}']   = df['Change'].shift(lag)
        df[f'RSI_lag{lag}']      = df['RSI'].shift(lag)
        df[f'Return{lag}']       = df['Close'].pct_change(lag)

    # 波動度特徵：計算 5 日短線與 20 日長線的波動比例
    df['Volatility20']     = df['STD20'] 
    df['Volatility5']      = df['Close'].rolling(5).std()
    df['Volatility_ratio'] = df['Volatility5'] / (df['Volatility20'] + 1e-8)

    # 滾動平均特徵：平滑化近 5 日的表現
    df['Change_mean_5d'] = df['Change'].rolling(5).mean()
    df['Change_std_5d']  = df['Change'].rolling(5).std()
    df['RSI_mean_5d']    = df['RSI'].rolling(5).mean()

    # 計算連續上漲/下跌天數：捕捉極端情緒與反轉可能
    up_mask   = (df['Movement'] == 1).astype(int)
    down_mask = (df['Movement'] == 0).astype(int)
    df['Consecutive_up'] = up_mask.groupby((up_mask == 0).cumsum()).cumsum()
    df['Consecutive_down'] = down_mask.groupby((down_mask == 0).cumsum()).cumsum()

    # ============= 2. K 棒實體與影線分析（價格型態） =============
    price_range = (df['High'] - df['Low']).replace(0, np.nan)
    # 收盤價在當日最高低點的位置：判斷收盤強度（0=最低點, 1=最高點）
    df["close_pos"] = (df["Close"] - df["Low"]) / (price_range + 1e-8)
    # 實體、上影線、下影線比例：辨識壓力與支撐區間
    df["body_size"]    = (df["Close"] - df["Open"]).abs() / (df["Open"] + 1e-8)
    df["upper_shadow"] = (df["High"] - df[["Open", "Close"]].max(axis=1)) / (df["Open"] + 1e-8)
    df["lower_shadow"] = (df[["Open", "Close"]].min(axis=1) - df["Low"]) / (df["Open"] + 1e-8)
    # 漲跌判斷
    df["is_bullish"] = (df["Close"] > df["Open"]).astype(int)
    df["is_bearish"] = (df["Close"] < df["Open"]).astype(int)
    # 大陽線/大陰線辨識
    df["marubozu_like"] = ((df["body_size"] > 0.02) & (df["upper_shadow"] < 0.005) & (df["lower_shadow"] < 0.005)).astype(int)

    # ============= 3. 布林通道 (Bollinger Bands) =============
    # 衡量價格波動範圍與極端超買賣區間
    bb_mid = df["Close"].rolling(20).mean()
    bb_std = df["Close"].rolling(20).std()
    df["BB_mid"]   = bb_mid
    df["BB_std"]   = bb_std
    df["BB_upper"] = bb_mid + 2 * bb_std
    df["BB_lower"] = bb_mid - 2 * bb_std
    # 價格在布林通道中的位置
    df["BB_pos"]   = (df["Close"] - df["BB_lower"]) / ((df["BB_upper"] - df["BB_lower"]).replace(0, np.nan) + 1e-8)

    # ============= 4. 多週期動量特徵 =============
    # 衡量不同時間框架下的報酬率 (3, 5, 10, 20 日)
    df["mom3"]  = df["Close"].pct_change(3)
    df["mom5"]  = df["Close"].pct_change(5)
    df["mom10"] = df["Close"].pct_change(10)
    df["mom20"] = df["Close"].pct_change(20)

    # ============= 5. 長期趨勢均線與黃金/死亡交叉 =============
    df["MA20"]  = df["Close"].rolling(20).mean()
    df["MA60"]  = df["Close"].rolling(60).mean()
    df["MA120"] = df["Close"].rolling(120).mean()
    df["Close_above_MA20"]  = (df["Close"] > df["MA20"]).astype(int)
    df["Close_above_MA60"]  = (df["Close"] > df["MA60"]).astype(int)
    df["Close_above_MA120"] = (df["Close"] > df["MA120"]).astype(int)
    df["MA5_MA20_cross"]  = (df["MA5"] > df["MA20"]).astype(int)
    df["MA20_MA60_cross"] = (df["MA20"] > df["MA60"]).astype(int)

    # ============= 6. 成交量量能特徵 =============
    if 'Volume' in df.columns:
        # 成交量變化率與量能爆發辨識
        df["vol_change"]   = df["Volume"].pct_change()
        vol_ma20           = df["Volume"].rolling(20).mean()
        df["vol_ratio20"]  = df["Volume"] / (vol_ma20 + 1e-8)
        df["vol_spike"]    = (df["vol_ratio20"] > 1.5).astype(int)

    return df

### 3. 特徵組合定義 (Feature Set Definition)

為了捕捉美股對台股的連動效應，我們定義了特定的特徵子集進行模型測試：
* **US_RAW**: 包含美股四大指數（NASDAQ, S&P 500, DJI, SOX）及台積電 ADR 的原始收盤價。
* **US_CHANGE**: 捕捉這些指數的單日漲跌變動率，藉此反應市場情緒的即時變化。
* **US_FEATURES**: 整合上述兩者，構建完整的美股影響矩陣。

In [6]:
US_RAW = ['NASDAQ', 'SPX', 'DJI', 'ADR', 'SOX']
US_CHANGE = ['NASDAQ_change', 'SPX_change', 'DJI_change']

US_FEATURES = US_RAW + US_CHANGE

### 4. 智能缺失值處理 (Smart NA Imputation)

由於特徵工程中使用了大量的滾動窗口（Rolling Windows）與滯後項（Lags），資料初期會產生必然的缺失值。為確保模型訓練的完整性，我們設計了 `fill_na_safely` 函式進行分類填補：

* **二元訊號類**: 如均線交叉、超買超跌訊號，缺失值統一補 `0`，表示「無特定事件發生」。
* **歷史滯後類 (Lags)**: 缺失值補 `-1`，藉此區分「無歷史資料」與「正常數值」。
* **震盪與動量類**: 如波動率、布林通道位置，補 `0` 以維持中性特徵。
* **連續數值類**: 採用 `ffill` (前向填補) 確保邏輯符合時間序列之真實性，避免使用到未來資訊。

In [7]:
def fill_na_safely(df: pd.DataFrame) -> pd.DataFrame:
    """
    智能缺失值填補函式 (Smart NA Imputation)：
    根據金融特徵的物理意義進行分類處理，避免簡單填補造成的資訊扭曲。
    """
    df = df.copy()

    for col in df.columns:
        # 跳過不需填補的欄位（時間、標籤、輔助標籤）
        if col in ['Date', 'Movement', 'Movement_tomorrow', 'is_train']:
            continue

        # ============= 1. 二元訊號與事件類特徵 =============
        # 邏輯：缺值通常發生在資料初期（還沒累積足夠天數），此時應視為「無事件發生」
        if (
            col.endswith('_cross') or      # 均線或指標交叉
            col.endswith('_spike') or      # 量能或波動爆發
            col.endswith('_breakout') or   # 價格突破
            ('MACD_' in col and 'cross' in col) or
            ('K_D_' in col and 'cross' in col) or
            col.endswith('_oversold') or   # 超跌訊號
            col.endswith('_overbought') or # 超買訊號
            col.startswith('is_') or       # 布林/K線狀態判斷
            col.endswith('_above_MA5') or
            col.endswith('_above_MA10') or
            col.endswith('_above_MA20') or
            col.endswith('_above_MA60') or
            col.endswith('_above_MA120') or
            col == 'vol_spike' or
            col == 'marubozu_like'
        ):
            df[col] = df[col].fillna(0) # 0 代表「無訊號」
            continue

        # ============= 2. 滯後類特徵 (Lag Features) =============
        # 邏輯：當資料尚未累積到足夠的 lag 天數時，使用 -1 作為標記。
        # 這能讓模型（特別是 Tree-based 模型）區分「無歷史」與「數值為 0」的差異。
        if 'lag' in col:
            df[col] = df[col].fillna(-1)
            continue

        # ============= 3. 滾動計算與震盪指標 =============
        # 邏輯：這類特徵反映的是相對位置或動量，初始值補 0 維持中性。
        if (
            col.endswith('_5d') or
            col.startswith('Volatility') or
            col.startswith('BB_') or
            col.startswith('mom') or
            col in ['close_pos', 'body_size', 'upper_shadow', 'lower_shadow', 'BB_pos']
        ):
            df[col] = df[col].fillna(0)
            continue

        # ============= 4. 變動率與回報率類型 =============
        # 邏輯：對於變動率類特徵，NaN 通常發生在第一筆，填 0 代表「無變動」。
        if (
            'change' in col.lower() or
            'return' in col.lower() or
            'ratio' in col.lower()
        ):
            df[col] = df[col].fillna(0)
            continue

        # ============= 5. 基礎數值欄位（如股價、指數） =============
        # 邏輯：採用時間序列最穩健的填補法。
        # 1. ffill (前向填補)：使用前一天的資料，這符合實際交易邏輯（不參考未來資訊）。
        # 2. bfill (後向填補)：若第一筆就缺值，才往後找。
        # 3. 0：最後的保底填補。
        if df[col].dtype != 'O':
            df[col] = df[col].fillna(method='ffill').fillna(method='bfill').fillna(0)

    return df

## ⚙️ 第三階段：自動化預處理流水線與時序對齊 (Automated Pipeline & Temporal Alignment)

為了確保模型訓練符合真實交易邏輯並提升開發效率，我們建構了 `preprocess_one_df` 整合函式。此階段重點處理時序資料中常見的「資料洩漏 (Data Leakage)」與「時差對齊」問題。

### 1. 預測目標定義 (Target Labeling)
* **目標變數**: `Movement_tomorrow`。
* **邏輯**: 透過 `.shift(-1)` 將「下一交易日」的漲跌標籤移至當前列，使模型學習以今日已知的特徵來預測未來的市場走向。

### 2. 美股市場時差校正 (Market Lag Correction)
* **背景**: 由於地理時差，台股當日開盤時，美股昨晚的收盤資訊方才確定。
* **對齊**: 針對 `US_FEATURES` 執行 `.shift(1)` 處理。這確保了模型在訓練過程中，美股特徵始終落後於標籤一日，嚴格遵守「僅使用歷史資訊預測未來」的回測原則。

### 3. 流水線自動化 (Pipeline Automation)
本函式封裝了以下完整步驟：
1. **時間排序**: 確保所有位移操作 (Shift) 基於正確的時序順序。
2. **特徵工程整合**: 調用 `enhance_features` 進行多維度指標擴展。
3. **缺失值處理**: 執行 `fill_na_safely` 進行邏輯化填補。
4. **清理無效樣本**: 剔除因 `shift` 操作產生的頭尾無效資料點（如最後一筆無法獲取明天答案的資料）。

In [8]:
def preprocess_one_df(df_raw: pd.DataFrame, is_train: int) -> pd.DataFrame:
    """
    單一資料集完整預處理流水線：
    包含：定義標籤、處理時差、特徵增強、缺失值清理。
    """
    df = df_raw.copy()
    df['is_train'] = is_train

    # 1. 時間排序：確保時間序列的連續性，這是所有 shift 操作的前提
    if 'Date' in df.columns:
        df = df.sort_values('Date').reset_index(drop=True)

    # 2. 定義預測目標 (Target)：
    # 使用 .shift(-1) 將「明天的漲跌」移到「今天」的列，讓模型學習用今天預測明天
    df['Movement_tomorrow'] = df['Movement'].shift(-1)

    # 3. 美股資訊延遲處理 (Market Lag Handling)：
    # 由於美股開盤時間晚於台股，台股當天開盤時只能參考到「美股昨晚」的表現。
    # 使用 .shift(1) 確保模型不會使用到「未來」的美股資訊，符合真實交易邏輯。
    for col in US_FEATURES:
        if col in df.columns:
            df[col] = df[col].shift(1)

    # 4. 特徵工程增強：執行自定義指標計算 (如 RSI, MACD, K線型態等)
    df = enhance_features(df)

    # 5. 智能 NA 填補：根據特徵屬性進行分類填補，最大化樣本利用率
    df = fill_na_safely(df)

    # 6. 清理無標籤樣本：
    # 由於 shift(-1)，最後一筆資料會沒有「明天的答案」，必須剔除以防干擾訓練。
    df = df.dropna(subset=['Movement_tomorrow']).reset_index(drop=True)

    # 7. 型態轉換：確保預測目標為分類格式 (int)
    df['Movement_tomorrow'] = df['Movement_tomorrow'].astype(int)

    return df

In [9]:
# ------------------------------------------------
# 4. 對 train / test 各自前處理（避免任何洩漏）
# ------------------------------------------------
train_proc = preprocess_one_df(train_raw, is_train=1)
test_proc  = preprocess_one_df(test_raw,  is_train=0)

## 🎯 第四階段：特徵篩選與數值清理 (Feature Selection & Cleaning)

在模型訓練前，我們進行最後的特徵校準，確保輸入數據的品質與穩定性。

### 1. 特徵自動化過濾
* **排除非預測欄位**: 移除 `Date` (時序標籤)、`Movement_tomorrow` (預測答案) 以及輔助用的 `is_train`。
* **類型篩選**: 自動遍歷所有欄位，僅保留數值型態 (Numerical) 特徵，排除字串或物件型態，確保符合 Scikit-learn 模型輸入規範。

### 2. 數值穩定化處理 (Numerical Stability)
* **無限大數值處理**: 金融指標在計算比例（如漲跌幅、成交量比率）時，偶爾會因分母為零產生 `inf`。我們將其統一替換為 `0`，以避免後續標準化 (Scaling) 時產生錯誤。

In [10]:

# ------------------------------------------------
# 5. 定義特徵欄位
# ------------------------------------------------
exclude_cols = ['Date', 'Movement', 'Movement_tomorrow', 'is_train']
features = [
    c for c in train_proc.columns
    if c not in exclude_cols and train_proc[c].dtype != 'O'
]

X_train_full = train_proc[features]
y_train_full = train_proc['Movement_tomorrow']

X_test = test_proc[features]
y_test = test_proc['Movement_tomorrow']

# 避免 inf
X_train_full = X_train_full.replace([np.inf, -np.inf], 0)
X_test       = X_test.replace([np.inf, -np.inf], 0)

In [11]:
# 挑選幾個關鍵欄位來對齊看看
check_cols = ['Date', 'Movement', 'Movement_tomorrow']
print("--- 檢查標籤對齊 (今日 vs 明日) ---")
print(train_proc[check_cols].head(5))

print("\n--- 檢查美股特徵延遲 (Market Lag) ---")
# 假設妳原本讀入的原始資料叫 train_raw
print("原始第二筆 DJI:", train_raw['DJI'].iloc[2])
print("處理後第二筆 DJI (應為 NaN 或已填補):", train_proc['DJI'].iloc[2])
print("處理後第三筆 DJI (應等於原始第二筆):", train_proc['DJI'].iloc[3])

print("\n--- 檢查數值品質 ---")
print(f"訓練集總樣本數: {len(train_proc)}")
print(f"是否有遺失值 (NA): {train_proc[features].isna().any().any()}")
print(f"是否有無窮大 (inf): {np.isinf(train_proc[features]).any().any()}")
print(f"特徵總數: {len(features)}")

--- 檢查標籤對齊 (今日 vs 明日) ---
        Date  Movement  Movement_tomorrow
0 2010-01-04         0                  0
1 2010-01-05         0                  1
2 2010-01-06         1                  0
3 2010-01-07         0                  0
4 2010-01-08         0                  1

--- 檢查美股特徵延遲 (Market Lag) ---
原始第二筆 DJI: 10573.68
處理後第二筆 DJI (應為 NaN 或已填補): 10572.02
處理後第三筆 DJI (應等於原始第二筆): 10573.68

--- 檢查數值品質 ---
訓練集總樣本數: 3680
是否有遺失值 (NA): False
是否有無窮大 (inf): False
特徵總數: 117


## ⚖️ 第五階段：時序驗證集切割 (Time-Series Split)

在金融預測模型中，樣本的先後順序至關重要。為了模擬真實的投資環境，我們捨棄傳統的隨機抽樣，改採**按時間順序切割**的驗證機制。

### 1. 切割邏輯
* **訓練集 (Train)**: 使用前 80% 的歷史數據（由 2010 年開始），用於模型參數的學習。
* **驗證集 (Validation)**: 使用剩餘 20% 的數據，用於在正式進入測試集前進行超參數微調（如 KNN 的 K 值）。
* **測試集 (Test)**: 保持獨立的最後一年數據，作為最終效能評估的基準。

### 2. 嚴禁洗牌 (No Shuffling)
本專案堅持「不洗牌」原則，確保訓練數據的時間點永遠早於驗證與測試數據，徹底杜絕** Look-ahead Bias (先行者偏差)**，使回測結果具備實戰參考價值。

In [12]:
# ------------------------------------------------
# 6. 時間序列切割 train / val（按時間順序）
# ------------------------------------------------
n_train = len(X_train_full)
split_idx = int(n_train * 0.8)

X_train = X_train_full.iloc[:split_idx]
y_train = y_train_full.iloc[:split_idx]

X_val   = X_train_full.iloc[split_idx:]
y_val   = y_train_full.iloc[split_idx:]

print(f"Train size = {len(X_train)}, Val size = {len(X_val)}, Test size = {len(X_test)}")

Train size = 2944, Val size = 736, Test size = 205


## 🧪 第六階段：超參數調校 - K值掃描 (Hyperparameter Tuning: K-Sweep)

為了尋找最適合本資料集的 KNN 模型參數，我們實作了 `sweep_k` 函式，針對不同的鄰居數 ($k$) 進行效能掃描。

### 1. 評估指標說明
除了基礎的 **Accuracy** 外，我們特別觀測以下指標：
* **Recall (Class 1)**: 在股票預測中，捕捉到「上漲」的能力至關重要。
* **Macro F1-score**: 考慮到漲跌類別的平衡性，確保模型在預測漲與跌時具備均等的效能，不偏向任何一方。

### 2. 標準化一致性
在掃描過程中，我們嚴格執行 `fit_transform` 於訓練集、`transform` 於驗證集，確保模型測試環境與未來實戰部署的資料縮放尺度一致。

### 3. 優化目標
我們的目標是選出一個 **Macro F1-score** 與 **Accuracy** 同時處於高位，且指標表現相對穩定的 $k$ 值，以提升模型的泛化能力。

In [13]:
def sweep_k(X_train, X_val, y_train, y_val, k_list=[3,5,7,11,15,21,31,41]):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    results = []

    print("===== Sweep K =====")
    for k in k_list:
        knn = KNeighborsClassifier(
            n_neighbors=k,
            weights='uniform'
        )
        knn.fit(X_train_scaled, y_train)
        y_pred = knn.predict(X_val_scaled)

        acc = accuracy_score(y_val, y_pred)
        rec1 = recall_score(y_val, y_pred)
        f1m  = f1_score(y_val, y_pred, average='macro')

        print(f"k={k:2d} | Acc={acc:.4f} | Recall1={rec1:.4f} | MacroF1={f1m:.4f}")

        results.append((k, acc, rec1, f1m))

    return results


### 📊 超參數調校結果分析

經過 $k \in \{3, 5, \dots, 41\}$ 的掃描，我們觀察到：
1. **最佳參數**: $k=11$ 在驗證集上取得了最佳的 Accuracy 與 Macro F1，在模型複雜度與泛化能力間取得了較好的平衡。
2. **最終策略**: 我們決定採用 $k=11$ 作為最終 KNN 模型的參數，並應用於獨立測試集進行效能驗證。

In [14]:
results = sweep_k(X_train, X_val, y_train, y_val)

===== Sweep K =====
k= 3 | Acc=0.4620 | Recall1=0.5050 | MacroF1=0.4581
k= 5 | Acc=0.4715 | Recall1=0.5176 | MacroF1=0.4673
k= 7 | Acc=0.5041 | Recall1=0.5704 | MacroF1=0.4977
k=11 | Acc=0.5408 | Recall1=0.6181 | MacroF1=0.5331
k=15 | Acc=0.5204 | Recall1=0.6106 | MacroF1=0.5108
k=21 | Acc=0.5054 | Recall1=0.6131 | MacroF1=0.4928
k=31 | Acc=0.4946 | Recall1=0.6357 | MacroF1=0.4750
k=41 | Acc=0.5122 | Recall1=0.6558 | MacroF1=0.4925


In [15]:
def run_basic_knn(X_train, X_val, y_train, y_val, n_neighbors=11):
    # 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # 建立最基礎的 KNN 模型
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights='uniform',   # 先用 uniform，之後可以試 distance
        metric='minkowski',  # p=2 → Euclidean
        p=2
    )

    # 訓練
    knn.fit(X_train_scaled, y_train)

    # 預測
    y_pred = knn.predict(X_val_scaled)

    # 評估
    acc = accuracy_score(y_val, y_pred)
    rec_up = recall_score(y_val, y_pred, pos_label=1)
    rec_down = recall_score(y_val, y_pred, pos_label=0)
    f1_macro = f1_score(y_val, y_pred, average='macro')

    print("===== Basic KNN (k = {}) 評估結果 =====".format(n_neighbors))
    print(f"Accuracy        : {acc:.4f}")
    print(f"Recall (Up=1)   : {rec_up:.4f}")
    print(f"Recall (Down=0) : {rec_down:.4f}")
    print(f"Macro F1        : {f1_macro:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_val, y_pred, digits=4))

    print("Confusion Matrix:")
    print(confusion_matrix(y_val, y_pred))

    return knn, scaler


In [16]:
knn_model, knn_scaler = run_basic_knn(X_train, X_val, y_train, y_val, n_neighbors=11)

===== Basic KNN (k = 11) 評估結果 =====
Accuracy        : 0.5408
Recall (Up=1)   : 0.6181
Recall (Down=0) : 0.4497
Macro F1        : 0.5331

Classification Report:
              precision    recall  f1-score   support

           0     0.5000    0.4497    0.4735       338
           1     0.5694    0.6181    0.5928       398

    accuracy                         0.5408       736
   macro avg     0.5347    0.5339    0.5331       736
weighted avg     0.5376    0.5408    0.5380       736

Confusion Matrix:
[[152 186]
 [152 246]]


In [17]:
X_test_scaled = knn_scaler.transform(X_test)
y_test_pred = knn_model.predict(X_test_scaled)
acc_test = accuracy_score(y_test, y_test_pred)
rec_up_test = recall_score(y_test, y_test_pred, pos_label=1)
rec_down_test = recall_score(y_test, y_test_pred, pos_label=0)
macro_f1_test = f1_score(y_test, y_test_pred, average='macro')

print("===== KNN Final Test Evaluation =====")
print(f"Accuracy     : {acc_test:.4f}")
print(f"Recall Up    : {rec_up_test:.4f}")
print(f"Recall Down  : {rec_down_test:.4f}")
print(f"Macro F1     : {macro_f1_test:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

===== KNN Final Test Evaluation =====
Accuracy     : 0.4585
Recall Up    : 0.4380
Recall Down  : 0.4881
Macro F1     : 0.4567

Classification Report:
              precision    recall  f1-score   support

           0     0.3761    0.4881    0.4249        84
           1     0.5521    0.4380    0.4885       121

    accuracy                         0.4585       205
   macro avg     0.4641    0.4631    0.4567       205
weighted avg     0.4800    0.4585    0.4624       205

Confusion Matrix:
[[41 43]
 [68 53]]


## PCA降維

In [18]:
from sklearn.decomposition import PCA
def run_pca_knn(
    X_train, X_val, y_train, y_val,
    n_components=30,   # PCA 主成分數量
    k_neighbors=11     # KNN 的 k
):
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score, recall_score, f1_score

    # ======= 1. 標準化（PCA & KNN 都需要）=======
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # ======= 2. PCA 降維 =======
    pca = PCA(n_components=n_components, random_state=42)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_val_pca   = pca.transform(X_val_scaled)

    # ======= 3. KNN =======
    knn = KNeighborsClassifier(
        n_neighbors=k_neighbors,
        weights='distance'
    )
    knn.fit(X_train_pca, y_train)

    # ======= 4. 預測 =======
    y_pred = knn.predict(X_val_pca)

    # ======= 5. 評估 =======
    acc = accuracy_score(y_val, y_pred)
    rec1 = recall_score(y_val, y_pred)
    rec0 = recall_score(y_val, y_pred, pos_label=0)
    f1m  = f1_score(y_val, y_pred, average='macro')

    #print(f"===== PCA + KNN (PCA={n_components}, k={k_neighbors}) =====")
    #print(f"Accuracy        : {acc:.4f}")
    #print(f"Recall (Up=1)   : {rec1:.4f}")
    #print(f"Recall (Down=0) : {rec0:.4f}")
    #print(f"Macro F1        : {f1m:.4f}")
    #print("\nClassification Report:")
    #print(classification_report(y_val, y_pred, digits=4))

    return {
        'acc': acc,
        'recall_up': rec1,
        'recall_down': rec0,
        'macro_f1': f1m
    }


In [19]:
def search_best_pca_knn(
    X_train, X_val, y_train, y_val,
    pca_list=[10, 20, 30, 40, 50],
    k_list=[3, 5, 11, 15, 21, 31, 41, 51]
):
    best_f1 = -1
    best_params = None
    best_model = None

    for pca_n in pca_list:
        for k in k_list:
            result = run_pca_knn(
                X_train, X_val, y_train, y_val,
                n_components=pca_n,
                k_neighbors=k
            )

            if result["macro_f1"] > best_f1:
                best_f1 = result["macro_f1"]
                best_params = (pca_n, k)
                best_model = result

    print("\n===== BEST MODEL =====")
    print(f"PCA components : {best_params[0]}")
    print(f"K neighbors    : {best_params[1]}")
    print(f"Best Macro F1  : {best_f1:.4f}")

    return best_params

In [20]:
best_pca, best_k = search_best_pca_knn(
    X_train, X_val, y_train, y_val,
    pca_list=[20,30,40],
    k_list=[5,11,21,31]
)


===== BEST MODEL =====
PCA components : 40
K neighbors    : 11
Best Macro F1  : 0.5129


In [21]:
pca = PCA(n_components=best_pca)
knn = KNeighborsClassifier(n_neighbors=best_k, weights='distance')

In [22]:
# 1. scale again
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled  = scaler.transform(X_test)

# 2. PCA again
pca = PCA(n_components=best_pca)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca  = pca.transform(X_test_scaled)

# 3. Train KNN
knn = KNeighborsClassifier(n_neighbors=best_k, weights='distance')
knn.fit(X_train_pca, y_train_full)

# 4. Predict test
y_test_pred = knn.predict(X_test_pca)

# 5. Evaluate test
acc_test = accuracy_score(y_test, y_test_pred)
rec_up_test = recall_score(y_test, y_test_pred, pos_label=1)
rec_down_test = recall_score(y_test, y_test_pred, pos_label=0)
macro_f1_test = f1_score(y_test, y_test_pred, average='macro')

print("===== KNN(after PCA) Final Test Evaluation =====")
print(f"Accuracy     : {acc_test:.4f}")
print(f"Recall Up    : {rec_up_test:.4f}")
print(f"Recall Down  : {rec_down_test:.4f}")
print(f"Macro F1     : {macro_f1_test:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, digits=4))

===== KNN(after PCA) Final Test Evaluation =====
Accuracy     : 0.5854
Recall Up    : 0.7438
Recall Down  : 0.3571
Macro F1     : 0.5465

Classification Report:
              precision    recall  f1-score   support

           0     0.4918    0.3571    0.4138        84
           1     0.6250    0.7438    0.6792       121

    accuracy                         0.5854       205
   macro avg     0.5584    0.5505    0.5465       205
weighted avg     0.5704    0.5854    0.5705       205



In [23]:
pca40 = PCA(n_components=40)
X_train_40 = pca40.fit_transform(X_train)
X_train_full_40 = pca40.fit_transform(X_train_full)
X_val_40   = pca40.transform(X_val)
X_test_40  = pca40.transform(X_test)

## 🧬 第七階段：進階特徵工程 - Cluster-Augmented KNN

為了進一步捕捉金融數據中的非線性結構與市場狀態，我們實作了 `run_cluster_knn` 函式。這是一種將非監督式學習與監督式學習結合的創新嘗試。

### 1. 核心邏輯：空間維度增強
* **KMeans 市場分群**: 透過 KMeans 算法，將 117 個原始特徵壓縮並分群。這相當於識別出特定的「市場型態」（Market Regimes），例如：特定技術指標組合下的震盪期或趨勢期。
* **特徵增強 (Feature Augmentation)**: 將 KMeans 產生的 Cluster Label 作為新的維度加入特徵矩陣中。這能引導 KNN 模型在尋找鄰居時，優先考慮屬於同一「市場狀態」的歷史樣本。

### 2. 模型防禦機制
* **時序一致性**: KMeans 僅在訓練集 (`X_train`) 上進行 `fit`，驗證集則採 `predict` 模式，嚴格遵守回測邏輯。
* **多維指標評估**: 考慮到金融預測的困難度，我們同時監控 **Recall Up** (看漲精準度) 與 **Recall Down** (看跌精準度)，確保模型不具備偏向性。

### 3. 優化動機
KNN 模型在維度過高時（118維）容易面臨「維度災難」。透過加入 Cluster Label，我們賦予了距離計算公式一個「邏輯錨點」，期待能藉此提升分類的解析度。

In [24]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score

def run_cluster_knn(X_train, X_val, y_train, y_val, 
                    n_clusters=7, n_neighbors=11):

    # 先用同一個 scaler 做 KMeans & KNN
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled   = scaler.transform(X_val)

    # KMeans clustering（只 fit 在 train）
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_train = kmeans.fit_predict(X_train_scaled)
    cluster_val   = kmeans.predict(X_val_scaled)

    # 將 cluster label 加入特徵
    X_train_aug = np.column_stack([X_train_scaled, cluster_train])
    X_val_aug   = np.column_stack([X_val_scaled, cluster_val])

    # 訓練 KNN
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights='uniform',
        metric='minkowski',
        p=2
    )
    knn.fit(X_train_aug, y_train)

    # 預測
    y_pred = knn.predict(X_val_aug)

    # 評估
    acc = accuracy_score(y_val, y_pred)
    rec_up = recall_score(y_val, y_pred, pos_label=1)
    rec_down = recall_score(y_val, y_pred, pos_label=0)
    macro_f1 = f1_score(y_val, y_pred, average='macro')

    print(f"\n===== ClusterKNN: clusters={n_clusters}, k={n_neighbors} =====")
    print(f"Accuracy     : {acc:.4f}")
    print(f"Recall Up    : {rec_up:.4f}")
    print(f"Recall Down  : {rec_down:.4f}")
    print(f"Macro F1     : {macro_f1:.4f}")

    return {
        "model": knn,
        "scaler": scaler,
        "kmeans": kmeans,
        "val_macro_f1": macro_f1
    }


### 📊 最終模型選定：Cluster-Augmented KNN (10 clusters, k=7)

經過多輪實驗對比，我們決定選用 **Cluster-Augmented KNN** 作為最終預測模型

In [25]:
for c in [3, 5, 7, 10, 15]:
    out = run_cluster_knn(
        X_train_40, X_val_40, y_train, y_val,
        n_clusters=c, 
        n_neighbors=11
    )


===== ClusterKNN: clusters=3, k=11 =====
Accuracy     : 0.5707
Recall Up    : 0.7312
Recall Down  : 0.3817
Macro F1     : 0.5488

===== ClusterKNN: clusters=5, k=11 =====
Accuracy     : 0.5720
Recall Up    : 0.7236
Recall Down  : 0.3935
Macro F1     : 0.5521

===== ClusterKNN: clusters=7, k=11 =====
Accuracy     : 0.5774
Recall Up    : 0.7387
Recall Down  : 0.3876
Macro F1     : 0.5557

===== ClusterKNN: clusters=10, k=11 =====
Accuracy     : 0.5679
Recall Up    : 0.7211
Recall Down  : 0.3876
Macro F1     : 0.5476

===== ClusterKNN: clusters=15, k=11 =====
Accuracy     : 0.5598
Recall Up    : 0.7211
Recall Down  : 0.3698
Macro F1     : 0.5374


## 🏁 第八階段：最終模型對決與測試集評估 (Final Model Evaluation)

在完成超參數調校後，我們將訓練集與驗證集合併，利用最大化的歷史數據重新訓練最佳化後的 **Cluster-Augmented KNN** 模型，並對獨立的測試集（2024年數據）進行最終效能驗證。

### 1. 最終模型配置
* **特徵處理**: 整合 `StandardScaler` 標準化與 117 維增強特徵。
* **分群策略**: 採用 `n_clusters=7` 的 KMeans 分群，為模型提供市場環境背景。
* **鄰居投票**: 設定 `n_neighbors=11`，在保持局部特徵敏感度的同時，具備足夠的噪訊過濾能力。

### 2. 多維評估指標
為了全面解析模型的實戰價值，我們不僅關注 **Accuracy**，更深入分析：
* **Confusion Matrix (混淆矩陣)**: 直觀展示「實際漲/跌」與「預測漲/跌」的分佈情況，檢查模型是否存在特定偏誤。
* **Recall Up/Down**: 衡量模型對市場上漲與下跌趨勢捕捉的敏感度。
* **Classification Report**: 提供精確度 (Precision) 與 F1-score，量化預測的綜合穩定性。

In [26]:
def final_cluster_knn(X_train_full, y_train_full, X_test, y_test,
                      n_clusters=7, n_neighbors=11):
    print("===== Training Final Cluster-KNN Model =====")

    # StandardScaler（fit 在全部 train）
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_full)
    X_test_scaled  = scaler.transform(X_test)

    # KMeans（fit 在全部 train）
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_train = kmeans.fit_predict(X_train_scaled)
    cluster_test  = kmeans.predict(X_test_scaled)

    # 將 cluster label 加入特徵
    X_train_aug = np.column_stack([X_train_scaled, cluster_train])
    X_test_aug  = np.column_stack([X_test_scaled, cluster_test])

    # KNN 訓練
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights='uniform',
        metric='minkowski',
        p=2
    )
    knn.fit(X_train_aug, y_train_full)

    # Test 預測
    y_pred_test = knn.predict(X_test_aug)

    # 評估
    acc = accuracy_score(y_test, y_pred_test)
    rec_up = recall_score(y_test, y_pred_test, pos_label=1)
    rec_down = recall_score(y_test, y_pred_test, pos_label=0)
    macro_f1 = f1_score(y_test, y_pred_test, average='macro')

    print("\n===== Final Cluster-KNN Test Evaluation =====")
    print(f"Accuracy     : {acc:.4f}")
    print(f"Recall Up    : {rec_up:.4f}")
    print(f"Recall Down  : {rec_down:.4f}")
    print(f"Macro F1     : {macro_f1:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_test, digits=4))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_test))

    return {
        "knn": knn,
        "kmeans": kmeans,
        "scaler": scaler,
        "macro_f1_test": macro_f1
    }


In [27]:
result = final_cluster_knn(
    X_train_full_40, y_train_full,
    X_test_40, y_test,
    n_clusters=7,
    n_neighbors=11
)

===== Training Final Cluster-KNN Model =====

===== Final Cluster-KNN Test Evaluation =====
Accuracy     : 0.6390
Recall Up    : 0.5455
Recall Down  : 0.7738
Macro F1     : 0.6390

Classification Report:
              precision    recall  f1-score   support

           0     0.5417    0.7738    0.6373        84
           1     0.7765    0.5455    0.6408       121

    accuracy                         0.6390       205
   macro avg     0.6591    0.6596    0.6390       205
weighted avg     0.6803    0.6390    0.6393       205

Confusion Matrix:
[[65 19]
 [55 66]]


## 📈 第九階段：生成漲跌機率與決策支援 (Probability Estimation)

在完成最終評估後，我們將模型應用於測試集，不僅生成類別預測，更計算其**上漲機率 (Probability)**。這能幫助我們識別預測的信心強度，作為實務交易的過濾器。

### 1. 跨模型的一致性轉換
* **標準化套用**: 嚴格使用訓練階段生成的 `scaler` 對測試集進行縮放，確保數據空間的一致性。
* **分群映射**: 利用訓練好的 `kmeans` 識別測試集樣本所屬的市場狀態 (Cluster Label)。

### 2. 從「分類」到「機率」 (predict_proba)
* **信心量化**: 透過 `predict_proba(X_test_aug)[:, 1]` 提取模型預測為「上漲 (Class 1)」的機率值。
* **決策門檻**: 
    * 若機率接近 1.0，代表鄰居一致看漲，信心度高。
    * 若機率接近 0.5，代表市場處於多空拉鋸，應謹慎入場。

In [28]:
# 1. 用 scaler + kmeans 處理 test 資料
X_test_scaled = result['scaler'].transform(X_test_40)
cluster_test = result['kmeans'].predict(X_test_scaled)

# 2. 增加 cluster label
X_test_aug = np.column_stack([X_test_scaled, cluster_test])

# 3. 產生上漲機率
y_pred_prob = result['knn'].predict_proba(X_test_aug)[:, 1]

## 💾 第十階段：生成最終預測報告 (Final Output Generation)

本階段將模型對 2025 年（測試集）的預測結果匯出為 CSV 格式。這份檔案將作為後續策略回測與直播演示的核心數據。

### 1. 輸出欄位定義
* **Date**: 對應測試集中的交易日期。
* **Predict**: 存儲模型預測之「上漲機率」。相較於二元分類 (0/1)，機率值能提供更細緻的決策參考，例如：機率高於 0.7 視為強烈買進訊號。

### 2. 數據封裝與持久化
透過將預測結果與時間戳記對齊，我們確保了模型輸出與原始股價數據的連動性，為明天的模型展示與績效分析奠定了基礎。

In [29]:
import os

# 1. 定義輸出路徑
output_dir = "../../../data/results/KNN"

# 2. 如果資料夾不存在，則自動建立
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"📁 已建立資料夾：{output_dir}")

# 3. 建立輸出 DataFrame
output_2025 = pd.DataFrame({
    "Date": test_proc["Date"].values,
    "Predict": y_pred_prob
})

# 4. 輸出成 CSV（放到指定資料夾下）
output_path = os.path.join(output_dir, "KNN_prediction_2022.csv")
output_2025.to_csv(output_path, index=False)

print(f"📄 已輸出：{output_path}")

📄 已輸出：../../../data/results/KNN\KNN_prediction_2022.csv


## 📂 第十一階段：資料集導出與多年份預測整合 (Data Export & Prediction Merging)

在本專案的最後階段，我們執行資料的物理切割與預測結果的自動化彙整，確保模型產出符合系統部署的要求。

### 1. 時序資料集切割
* **訓練集與測試集**: 嚴格按照年份（ex:2022年為界）進行切割，確保驗證過程符合現實中的時間推移邏輯。
* **持久化存儲**: 將切割後的 `2022_train.csv` 與 `2022_test.csv` 導出，便於後續進行特定年份的深度回測。
> ⚠️ **Stacking 預備作業提醒**：
> 為避免資料洩漏（Data Leakage），在生成 Stacking 所需的 Meta-features 時，必須採取「滾動式預測」。
> 1. 先生成特定年份（如 2020）的 `train/test` 物理切片。
> 2. 將預處理管線的輸入指向該切片，重新訓練並輸出 `KNN_prediction_2020.csv`。
> 3. 逐年重複此動作（2020-2024），最後再透過 `glob` 執行自動化合併。

In [216]:
df = pd.read_csv("../../../data/processed/train.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

In [217]:
df_train_2020 = df[df["Date"].dt.year < 2024]

In [218]:
df_test_2020 = df[df["Date"].dt.year == 2024]

In [219]:
# 1. 定義處理後資料的存放路徑
processed_dir = "../../../data/processed"

# 2. 檢查並建立資料夾
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# 3. 定義完整的檔案路徑
train_path_2020 = os.path.join(processed_dir, "2024_train.csv")
test_path_2020  = os.path.join(processed_dir, "2024_test.csv")

# 4. 儲存檔案
df_train_2020.to_csv(train_path_2020, index=False)
df_test_2020.to_csv(test_path_2020, index=False)

print(f"✅ 已儲存至：{processed_dir}")

✅ 已儲存至：../../data/processed


### 2. 自動化預測彙整 (Automated Result Merging)
* **批次讀取**: 利用 `glob` 模組自動偵測目錄下所有符合 `KNN_prediction_*.csv` 模式的檔案，提升代碼的擴充性。
* **時序合併**: 將不同年份的預測機率進行垂直合併（Concat），並重新執行時間排序，確保最終輸出的 `KNN_prediction_all.csv` 具備完整且連續的時間序列。
* **最終產出**: 此彙整檔案將作為stacking的依據。

In [604]:
import glob

# 1. 定義路徑
input_pattern = "../../../data/results/KNN/KNN_prediction_*.csv" # 從 results 讀取各年份檔案
output_dir = "../../../data/results/KNN"
output_file = "KNN_prediction_all.csv"

# 2. 檢查並建立輸出資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 3. 尋找所有預測檔案
files = sorted(glob.glob(input_pattern))

if not files:
    print("❌ 找不到預測檔案，請確認 data/results/ 下是否有檔案。")
else:
    dfs = []
    for f in files:
        df = pd.read_csv(f)
        dfs.append(df)

    # 合併數據
    merged = pd.concat(dfs, ignore_index=True)

    # 若有 Date 欄位則進行排序
    if "Date" in merged.columns:
        merged["Date"] = pd.to_datetime(merged["Date"])
        merged = merged.sort_values("Date").reset_index(drop=True)

    # 4. 輸出到 data/processed/KNN/
    final_save_path = os.path.join(output_dir, output_file)
    merged.to_csv(final_save_path, index=False)

    print(f"✔ 已輸出最終合併檔：{final_save_path}")

✔ 已輸出最終合併檔：../../data/results/KNN\KNN_prediction_all.csv
